In [1]:
import pandas as pd
import numpy as np
from environment import Environment
from pandas.tseries.offsets import MonthEnd
import instrument
from portfolio import Portfolio
from risk_parity import RiskParity


%load_ext autoreload
%autoreload 2

In [2]:
from historical import HistoricalData
from pandas._libs.tslibs.timestamps import Timestamp, Timedelta

path = "data\\"
env_df = pd.read_csv(path + "Consolidated.csv", header = [0,1], index_col=[0])
targets_df = pd.read_csv(path + "Targets.csv", index_col=[0])
features_df = pd.read_csv(path + "Features.csv", header = [0,1], index_col=[0])
exps = pd.read_csv('data\\ETF_info.csv', index_col=0).loc[env_df[['EQ', 'EM', 'RE', 'FI']].columns.droplevel(), 'Expense Ratio'].astype(float)
data = HistoricalData(env_df, features_df, targets_df)

date = Timestamp('2014-03-01') + MonthEnd(0)
date_end = date + MonthEnd(60)
env_now = Environment(*data.get_env_args(date))
rp = RiskParity(data)


### Initializing stuff

In [3]:
def dumb_optimize(env: Environment):
    w_dict = {}
    n_assets = len(env.prices.items())
    for asset, price in env.prices.items():
        w_dict[asset] = 1/n_assets
    return w_dict

usd = 200000/env_now.fx['CAD']
w_dict = rp.get_weights_ac(date, asset_classes=np.array(['FI', 'RE', 'EM']))
# w_dict = dumb_optimize(env_now)
pos_dict = Portfolio.weights_to_pos(w_dict, env_now, usd)
pos_dict = Portfolio.etf_dict_from_names(pos_dict)
icash = instrument.Cash('USD')
pos_dict[icash] = 0

In [4]:
my_pf = Portfolio(pos_dict)
etf_w = 0.98
etf_total_val= my_pf.calc_value(env_now)*etf_w
opt_total_val = my_pf.calc_value(env_now)*(1-etf_w)

my_pf.rebalance(env_now, Portfolio.weights_to_pos(w_dict, env_now, etf_total_val), exps, time_past=0)


C = 1 + 0.2/12
K1 = C - 0.03
K2 = C
K3 = C + 0.03
ttm=2

specs = [{'name':'SPY Call 1', 'ccy':'USD', 'is_call':1, 'ul':'SPY US Equity'},
         {'name':'SPY Call 2', 'ccy':'USD', 'is_call':1, 'ul':'SPY US Equity'},
         {'name':'SPY Call 3', 'ccy':'USD', 'is_call':1, 'ul':'SPY US Equity'}
         ]

cost = Portfolio.get_opt_strategy_price(env_now, specs, [1, -2, 1], [K1,K2,K3], ttm=ttm)
nopt = opt_total_val/cost
ofee = my_pf.buy_options(env_now, specs, [nopt, -nopt*2, nopt], [K1,K2,K3], ttm=ttm)

### Set up performance df

In [5]:
perf_df = pd.DataFrame(index = pd.date_range(start=date, end=date_end, freq='M'))
perf_df['USD Value'] = 0
perf_df['CAD Value'] = 0
perf_df['Return'] = 0
perf_df['PNL'] = 0
perf_df['Injection'] = 0
perf_df['FX'] = env_now.fx['CAD']
perf_df['Option Fees'] = ofee
perf_df['ETF Fees'] = 0
perf_df['ETF Expenses'] = 0

### Run sim/backtest

In [6]:
perf_df.loc[date, 'USD Value'] = my_pf.calc_value(env_now)
perf_df.loc[date, 'CAD Value'] = my_pf.calc_value(env_now, ccy='CAD')
perf_df.loc[date, 'Return'] = np.nan
perf_df.loc[date, 'PNL'] = np.nan

while date < date_end:
    
    date = date + MonthEnd(1)
    env_now = Environment(*data.get_env_args(date))
    w_dict = rp.get_weights_ac(date, asset_classes=np.array(['FI', 'RE', 'EM']))
    
    
    etf_total_val= my_pf.calc_value(env_now)*etf_w
    opt_total_val = my_pf.calc_value(env_now)*(1-etf_w)
    
    perf_df.loc[date, 'USD Value'] = my_pf.calc_value(env_now)
    perf_df.loc[date, 'CAD Value'] = my_pf.calc_value(env_now, ccy='CAD')

    perf_df.loc[date, 'FX'] = env_now.fx['CAD']
    
    if (date.month == 8) or (date.month == 2):
        my_pf.pf_units[my_pf.get_cash('USD')] += 20000/env_now.fx['CAD']
        perf_df.loc[date, 'Injection'] = 20000/env_now.fx['CAD']
        
    perf_df.loc[date, 'Return'] = perf_df.loc[date, 'USD Value']/(perf_df.loc[date + MonthEnd(-1), 'USD Value'] + perf_df.loc[date + MonthEnd(-1), 'Injection']) - 1
    perf_df.loc[date, 'PNL'] = perf_df.loc[date, 'USD Value'] - (perf_df.loc[date + MonthEnd(-1), 'USD Value'] + perf_df.loc[date + MonthEnd(-1), 'Injection'])
    
    efee, eexps = my_pf.rebalance(env_now, Portfolio.weights_to_pos(w_dict, env_now, etf_total_val), exps)
    ofee = my_pf.sell_options(env_now)
    
    # re-buy options
    
    cost = Portfolio.get_opt_strategy_price(env_now, specs, [1, -2, 1], [K1,K2,K3], ttm=ttm)
    nopt = opt_total_val/cost    
    ofee += my_pf.buy_options(env_now, specs, [nopt, -nopt*2, nopt], [K1,K2,K3], ttm=ttm)
    # record fees
    perf_df.loc[date, 'ETF Fees'] = efee
    perf_df.loc[date, 'ETF Expenses'] = eexps
    perf_df.loc[date, 'Option Fees'] = ofee
    
    #####
    # my_pf.get_forw_risk(env_now: Environment, dist: Distribution, N=1000) vector of 1000 simulated forward pnls
    # -> self.emp_dist = xxxx
    # -> my_pf.calc_var()
    # -->   return -np.percentile(self.emp_dist, 5)
    # -> my_pf.calc_cvar()
    
    
    #####
    

### Evaluate performance

In [7]:
perf_df

,USD Value,CAD Value,Return,PNL,Injection,FX,Option Fees,ETF Fees,ETF Expenses
2014-03-31,180790.988689,199765.002952,NaN,NaN,0.000000,1.10495,153.276635,0.00,0.000000
2014-04-30,186150.851021,204051.116855,0.029647,5359.862332,0.000000,1.09616,306.379943,24.75,33.071057
2014-05-31,194270.201132,210697.689340,0.043617,8119.350111,0.000000,1.08456,296.153381,0.00,33.673716
2014-06-30,201247.484334,214741.128159,0.035915,6977.283202,0.000000,1.06705,285.753802,9.90,35.608070
2014-07-31,200078.051851,218209.124909,-0.005811,-1169.432484,0.000000,1.09062,320.299580,59.40,36.207474
2014-08-31,211761.797043,230365.070913,0.058396,11683.745192,18384.887622,1.08785,332.312871,9.90,37.108292
2014-09-30,221906.666812,248491.085496,-0.035803,-8240.017854,0.000000,1.11980,348.603246,0.00,35.909690
2014-10-31,233795.551932,263394.068806,0.053576,11888.885120,0.000000,1.12660,376.746652,14.85,40.718521
2014-11-30,243056.659276,277471.051662,0.039612,9261.107344,0.000000,1.14159,353.688755,0.00,42.053841
2014-12-31,241377.350970,280499.792015,-0.006909,-1679.308305,0.000000,1.16208,374.639896,44.55,42.771605


In [8]:
irr_values = np.array([-perf_df['USD Value'].iloc[0]] + (-perf_df['Injection'].values).tolist())
irr_values[-1] += perf_df['USD Value'].iloc[-1]
np.irr(irr_values)*12

0.19477469311795392

In [9]:
m = perf_df['Return'].mean()*12
s = perf_df['Return'].std()*np.sqrt(12)
var = -np.percentile(perf_df['Return'].dropna(), 5)*np.sqrt(12)
cvar = -perf_df['Return'][perf_df['Return']<np.percentile(perf_df['Return'].dropna(), 5)].mean()*np.sqrt(12)
print("Mean: {:.1f}%\nStd: {:.1f}%\nVaR: {:.1f}%\nCVaR: {:.1f}%".format(m*100, s*100, var*100, cvar*100))
print("Sharpe: {:.1f}".format(m/s))

Mean: 20.1%
Std: 9.0%
VaR: 10.4%
CVaR: 12.5%
Sharpe: 2.2


In [10]:
import matplotlib.pyplot as plt
perf_df['Return'].dropna().hist()
plt.show()

<Figure size 640x480 with 1 Axes>